# Training

In [ ]:
import os  # Модуль для взаємодії з операційною системою
import numpy as np  # Бібліотека для роботи з масивами даних
import tensorflow as tf  # Бібліотека для роботи з нейронними мережами
from keras.preprocessing.image import img_to_array, ImageDataGenerator  # Модулі для обробки зображень
from keras.models import Sequential, save_model  # Модулі для створення та збереження моделі
from keras.layers import Dense, Flatten, Activation, Dropout, Conv2D, MaxPooling2D  # Шари нейронних мереж
from keras.utils import to_categorical  # Утиліти для роботи з мітками класів
from keras.optimizers import SGD  # Оптимізатор навчання
from sklearn.model_selection import train_test_split  # Модуль для розбиття даних на навчальний та тестовий набори
import matplotlib.pyplot as plt  # Бібліотека для візуалізації даних
import time  # Модуль для роботи з часом
from PIL import Image  # Модуль для роботи з зображеннями
import cv2  # Бібліотека для обробки зображень

In [ ]:
import warnings  # Модуль для управління попередженнями
warnings.filterwarnings("ignore", category=UserWarning)  # Ігнорувати попередження UserWarning

In [ ]:
data = []  # Створення порожнього списку для даних
labels = []  # Створення порожнього списку для міток

nb_classes = 43  # Кількість класів
batch_size = 32  # Розмір партії даних
nb_epoch = 30  # Кількість епох навчання
img_rows, img_cols = 32, 32  # Розміри зображення
img_channels = 3  # Кількість каналів зображення (RGB)

for num in range(0, nb_classes):  # Ітерація по кожному класу
    path = os.path.join('drive/MyDrive/GoogleColab/data/Train', str(num))  # Шлях до теки з зображеннями для певного класу
    imagePaths = os.listdir(path)  # Отримання списку шляхів до зображень в цій теці
    for img in imagePaths:  # Ітерація по кожному зображенню
        image = Image.open(path + '/' + img)  # Відкриття зображення
        image = cv2.resize(img_to_array(image), (img_rows, img_cols), interpolation=cv2.INTER_LINEAR)  # Зміна розміру зображення
        data.append(image)  # Додавання зображення до списку даних
        labels.append(num)  # Додавання мітки до списку міток

data = np.array(data).astype('float32')  # Конвертація списку даних у масив numpy та зміна типу даних на float32
data /= 255  # Нормалізація даних (перетворення значень в діапазон від 0 до 1)

labels = to_categorical(np.array(labels), nb_classes)  # Перетворення міток на one-hot вектори

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)  # Розбиття даних на тренувальний та тестовий набори

In [ ]:
y_train_classes = np.argmax(y_train, axis=1)  # Отримання числових міток класів для тренувального набору даних
unique_classes, counts = np.unique(y_train_classes, return_counts=True)  # Знаходження унікальних класів та їх кількості в тренувальному наборі даних

plt.figure(figsize=(12, 8))  # Створення нового графіку з заданими розмірами
plt.bar(unique_classes, counts, align='center', alpha=0.7, color='orange')  # Створення стовпчикової діаграми: унікальні класи на вісі x, кількість елементів на вісі y
plt.xlabel('Class')  # Підпис вісі x
plt.ylabel('Number of elements')  # Підпис вісі y
plt.title('Histogram of class distribution')  # Заголовок графіку
plt.xticks(np.arange(nb_classes))  # Задання позначок на вісі x відповідно до унікальних класів
plt.show()  # Показ графіку

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10,  # Діапазон обертання в градусах
    width_shift_range=0.1,  # Діапазон зсуву по ширині
    height_shift_range=0.1,  # Діапазон зсуву по висоті
    shear_range=0.2,  # Діапазон зсуву
    zoom_range=0.2,  # Діапазон зуму
    horizontal_flip=True,  # Випадкова горизонтальне відображення
    fill_mode='nearest'  # Режим заповнення пікселя за межами введення
)

def augment_data(data, labels, min_samples):
    augmented_data = []  # Створення порожнього списку для зберігання розширених зображень
    augmented_labels = []  # Створення порожнього списку для зберігання міток розширених зображень

    for class_label in range(nb_classes):  # Перебір класів
        class_indices = np.where(labels.argmax(axis=1) == class_label)[0]  # Отримання індексів зображень з певним класом
        num_samples = len(class_indices)  # Кількість зображень у поточному класі

        if num_samples < min_samples:  # Якщо кількість зображень у класі менше мінімальної кількості
            num_augmented_samples = min_samples - num_samples  # Кількість зображень, які потрібно розширити
            random_indices = np.random.choice(class_indices, num_augmented_samples)  # Випадковий вибір індексів для розширення

            for index in random_indices:  # Перебір випадкових індексів
                img = data[index]  # Вибір зображення з випадковим індексом
                img = img.reshape((1,) + img.shape)  # Зміна форми зображення для генерації партії зображень
                augmented_images = []  # Створення порожнього списку для зберігання розширених зображень

                for batch in datagen.flow(img, batch_size=1):  # Підвибірка розширених зображень
                    augmented_images.append(batch[0])  # Додавання розширеного зображення до списку
                    if len(augmented_images) >= 1:  # Якщо кількість розширених зображень достатня
                        break  # Зупинка генерації додаткових зображень

                for augmented_image in augmented_images:  # Перебір розширених зображень
                    augmented_data.append(augmented_image)  # Додавання розширеного зображення до списку
                    augmented_labels.append(labels[index])  # Додавання мітки відповідного оригінального зображення

    augmented_data = np.array(augmented_data)  # Конвертування списку розширених зображень у масив NumPy
    augmented_labels = np.array(augmented_labels)  # Конвертування списку міток розширених зображень у масив NumPy

    return np.concatenate((data, augmented_data)), np.concatenate((labels, augmented_labels))  # Повертається об'єднаний масив оригінальних та розширених зображень, а також об'єднаний масив міток

X_train, y_train = augment_data(X_train, y_train, min_samples=500)

In [ ]:
# Отримання індексу класу з найвищою ймовірністю для кожного зображення у тренувальному наборі
y_train_classes = np.argmax(y_train, axis=1)
# Підрахунок кількості зображень у кожному класі
unique_classes, counts = np.unique(y_train_classes, return_counts=True)
# Створення нового графіку
plt.figure(figsize=(12, 8))
# Побудова гістограми
plt.bar(unique_classes, counts, align='center', alpha=0.7, color='orange')
# Підпис осі x
plt.xlabel('Class')
# Підпис осі y
plt.ylabel('Number of elements')
# Заголовок графіку
plt.title('Histogram of class distribution')
# Встановлення міток на осі x для кожного класу
plt.xticks(np.arange(nb_classes))
# Відображення графіку
plt.show()

In [ ]:
# Створення моделі Sequential
model = Sequential()
# Додавання першого шару згорткових нейронів
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3), activation='relu'))
# Додавання другого шару згорткових нейронів
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
# Додавання шару пулінгу для зменшення розмірності
model.add(MaxPooling2D(pool_size=(2, 2)))
# Додавання шару Dropout для запобігання перенавчання
model.add(Dropout(0.25))
# Додавання третього шару згорткових нейронів
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
# Додавання четвертого шару згорткових нейронів
model.add(Conv2D(64, (3, 3), activation='relu'))
# Додавання ще одного шару пулінгу
model.add(MaxPooling2D(pool_size=(2, 2)))
# Додавання ще одного шару Dropout
model.add(Dropout(0.25))
# Розгладжування вхідних даних перед подачею на повнозв'язний шар
model.add(Flatten())
# Додавання повнозв'язного шару з 512 нейронами
model.add(Dense(512, activation='relu'))
# Додавання ще одного шару Dropout
model.add(Dropout(0.5))
# Додавання повнозв'язного шару з кількістю нейронів, рівною кількості класів
model.add(Dense(nb_classes, activation='softmax'))
# Компіляція моделі з використанням функції втрат категоріальної крос-ентропії та оптимізатора Adam
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Початок вимірювання часу тренування
start_time = time.time()
# Навчання моделі з використанням навчальних даних
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=nb_epoch,
          validation_split=0.1,
          shuffle=True,
          verbose=2)

# Кінець вимірювання часу тренування
end_time = time.time()
# Оцінка ефективності моделі на тестових даних
scores = model.evaluate(X_test, y_test, verbose=0)
# Розрахунок тривалості тренування
hours, remainder = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(remainder, 60)
print(f"\nTraining period: {int(hours)}h:{int(minutes)}m:{int(seconds)}s")
# Виведення точності моделі на тестових даних
print("Accuracy on test data: %.2f%%" % (scores[1]*100))
# Збереження моделі на диск
save_model(model, 'my_model.h5', save_format='h5')


# Recognition

In [ ]:
# Завантаження необхідних бібліотек
from keras.models import load_model
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Завантаження моделі з файлу my_model.h5
model = load_model('my_model.h5')

In [ ]:
# Визначаємо шлях до теки з зображеннями та вибираємо всі файли .jpg
folder_path = "drive/MyDrive/GoogleColab/data/Images/Test"
image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".jpg")]

# Створюємо пустий список для зберігання масивів зображень
images_array = []

# Проходимося по кожному файлу .jpg у текі
for image_file in image_files:
    # Відкриваємо кожне зображення
    with Image.open(image_file) as original_image:
        # Змінюємо розмір зображення на 32x32 та перетворюємо його у масив
        resized_image = original_image.resize((32, 32), Image.BICUBIC)
        img_array = np.asarray(resized_image, dtype="float32") / 255.0
        # Додаємо масив зображення до списку images_array
        images_array.append(img_array)

In [ ]:
# Перетворення списку масивів зображень у масив NumPy
new_images_array = np.array(images_array)

# Визначення кількості стовпців та рядків для розміщення зображень у сітці
num_cols = 10
num_images = len(new_images_array)
num_rows = int(np.ceil(num_images / num_cols))

# Створення графіку та візуалізація зображень у вигляді сітки
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 5))

for i, ax in enumerate(axes.flatten()):
    if i < num_images:
        ax.imshow(new_images_array[i])
        ax.axis('off')
    else:
        ax.axis('off')

plt.show()

In [ ]:
# Список назв класів
class_names = ['3.29 - Обмеження максимальної швидкості 20', '3.29 - Обмеження максимальної швидкості 30', '3.29 - Обмеження максимальної швидкості 50',
               '3.29 - Обмеження максимальної швидкості 60', '3.29 - Обмеження максимальної швидкості 70', '3.29 - Обмеження максимальної швидкості 80',
               '3.30 - аКінець обмеження максимальної швидкості 80', '3.29 - Обмеження максимальної швидкості 100', '3.29 - Обмеження максимальної швидкості 120',
               '3.25 - Обгін заборонено', '3.27 - Обгін вантажним автомобілям заборонено', '1.22 - Перехрещення з другорядною дорогою',
               '2.30 - Головна дорога', '2.1 - Дати дорогу', '2.2 - Проїзд без зупинки заборонено', '3.1 - Рух заборонено', '3.30 - Рух вантажних автомобілів заборонено',
               '3.21 - В’їзд заборонено', '1.39 - Аварійно-небезпечна ділянка (інша небезпека)', '1.2 - Небезпечний поворот ліворуч',
               '1.1 - Небезпечний поворот праворуч', '1.3.2 - Декілька поворотів', '1.10 - Нерівна дорога', '1.13 - Слизька дорога', '1.5.2 - Звуження дороги',
               '1.37 - Дорожні роботи', '1.24 - Світлофорне регулювання', '1.32 - Пішохідний перехід', '1.33 - Діти', '1.34 - Виїзд велосипедистів',
               'Небезпека снігу чи льоду', '1.36 - Дикі тварини', '3.42 - Кінець усіх заборон і обмежень', '4.2 - Рух праворуч', '4.3 - Рух ліворуч', '4.1 - Рух прямо',
               '4.4 - Рух прямо або праворуч', '4.5 - Рух прямо або ліворуч', '4.7 - Об’їзд перешкоди з правого боку', '4.8 - Об’їзд перешкоди з лівого боку',
               '4.10 - Круговий рух', '3.26 - Кінець заборони обгону', '3.28 - Кінець заборони обгону вантажним автомобілям']

# Передбачення класів для нових зображень
predictions = model.predict(new_images_array)
predicted_classes = np.argmax(predictions, axis=1)

In [ ]:
# Визначення кількості стовпчиків та рядків для розміщення зображень
num_cols = 1
num_images = len(new_images_array)
num_rows = int(np.ceil(num_images / num_cols))

# Створення фігури та набору підграфіків
fig, axes = plt.subplots(num_rows, num_cols, figsize=(25, 15))

# Цикл по всіх підграфіках
for i, ax in enumerate(axes.flatten()):
    # Перевірка, чи індекс не перевищує кількість зображень
    if i < num_images:
        # Відображення зображення
        ax.imshow(new_images_array[i])
        # Вимкнення вісей координат
        ax.axis('off')
        # Додавання тексту з передбаченим класом під зображенням
        ax.text(0.5, -0.15, f"{class_names[predicted_classes[i]]}", size=10, ha="center", transform=ax.transAxes)
    else:
        ax.axis('off')

# Показ графіку
plt.show()